- https://python.langchain.com/docs/modules/agents/how_to/custom_agent

In [16]:
# Load the LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [17]:
# Define Tools
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word"""
    return len(word)

get_word_length.invoke("abc")

3

In [18]:
tools = [get_word_length]

In [19]:
# Create Prompt
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very powerful assistant. but don't know current event"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [20]:
# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

In [21]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [22]:
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
list(agent_executer.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...



Invoking: `get_word_length` with `{'word': 'eudca'}`


5There are 5 letters in the word "eudca".

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ZVe8sg8xS6PXN80oTGej8COx', 'function': {'arguments': '{\n  "word": "eudca"\n}', 'name': 'get_word_length'}, 'type': 'function'}]})], tool_call_id='call_ZVe8sg8xS6PXN80oTGej8COx')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ZVe8sg8xS6PXN80oTGej8COx', 'function': {'arguments': '{\n  "word": "eudca"\n}', 'name': 'get_word_length'}, 'type': 'function'}]})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_ZVe8sg8xS6PXN80oTGej8COx', 'function': {'arguments': '{

In [24]:
llm.invoke("How many letters in the word educa")

AIMessage(content='There are 6 letters in the word "educa".')